In [36]:
import pandas as pd
import numpy as np

In [37]:
data = pd.read_csv('melb_data.csv')

In [38]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [39]:
y = data.Price

In [54]:
x = data.drop(['Price'], axis = 1)

In [55]:
from sklearn.model_selection import train_test_split

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x,y ,train_size = .8,test_size = .2 , random_state = 0)

In [78]:
cat_col = x_train.select_dtypes(include=['object']).columns.tolist()
num_col = x_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

num_col

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [79]:
categorical_cols = [cname for cname in x_train.columns if x_train[cname].nunique() < 10 and 
                        x_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

print(numerical_cols)
print(categorical_cols)

['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
['Type', 'Method', 'Regionname']


In [80]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [84]:
num_procc = SimpleImputer(strategy = "constant")

cat_procc = Pipeline(steps= [('imputer',  SimpleImputer(strategy = 'most_frequent')),
                             ('encoder' , OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers= [('Numerical' , num_procc, num_col),
                                               ("Categorical" , cat_procc, cat_col)])

model = RandomForestRegressor(n_estimators = 50, random_state = 150 )

my_pipeline = Pipeline( steps= [("Preprocess" , preprocessor),
                               ('Model', model)])

my_pipeline.fit(x_train, y_train)

Pipeline(steps=[('Preprocess',
                 ColumnTransformer(transformers=[('Numerical',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('Categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                            

In [85]:
pred = my_pipeline.predict(x_test)

In [86]:
from sklearn.metrics import mean_absolute_error as mea

err = mea(y_test, pred)

err

161778.67957290134

# Using Cross Validation


In [91]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(my_pipeline, x, y, cv = 5, scoring = 'neg_mean_absolute_error')

In [97]:
-1 * score

array([202795.33818115, 187381.4553461 , 183153.55675258, 151531.59622975,
       159464.80122239])

In [98]:
final_score = -1 * score.mean()

In [104]:
print('Accuracy', final_score)

Accuracy 176865.34954639172
